<a href="https://colab.research.google.com/github/VMBoehm/SDSS_PAE/blob/main/LSTM_AE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fitting the Density Estimator

In [1]:
## todo add forked repo as origin or upstream
## check paths
import numpy as np

In [2]:
from sig_gis import *

In [3]:
from sig_gis.GIS import *

In [4]:
label           = 'galaxies_bins1000_wl3388-8318'
label_          = label+'_minz005_maxz036_minSN50'
label_2         = label_+'_10_fully_connected'

In [5]:
encoded_train, encoded_valid = np.load('/home/nessa/Documents/codes/SDSS_PAE/encoded_%s.npy'%label_2, allow_pickle=True)

In [6]:
# res_fluxes, res_masks, res_inv_vars = np.load('prepped_data_spectra_%s.npy'%label_,allow_pickle=True)

# redshifts, SNs, ras, decs, sublabel = np.load('prepped_data_prop_%s.npy'%label_,allow_pickle=True)

In [7]:
model = train_GIS(torch.tensor(encoded_train), data_validate=torch.tensor(encoded_valid), alpha=(0.5,0.5))

/home/nessa/miniconda3/envs/pytorch/lib/python3.7/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 8000). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at  /opt/conda/conda-bld/pytorch_1623448224956/work/c10/cuda/CUDAFunctions.cpp:115.)
  return torch._C._cuda_getDeviceCount() > 0
/home/nessa/Documents/codes/SIG_GIS/sig_gis/SINF.py:480: UserWarning: torch.qr is deprecated in favor of torch.linalg.qr and will be removed in a future PyTorch release.
The boolean parameter 'some' has been replaced with a string parameter 'mode'.
Q, R = torch.qr(A, some)
should be replaced with
Q, R = torch.linalg.qr(A, 'reduced' if some else 'complete') (Triggered internally at  /opt/conda

Fit A: Time: 15.677074670791626 Wasserstein Distance: [2.7009077072143555, 1.3881100416183472, 1.322676181793213, 1.293419599533081, 0.9838410019874573, 0.9396671652793884, 0.8025850653648376, 0.4535483121871948]
Fit spline Time: 0.4404752254486084
logp: -10.131624221801758 -10.167842864990234 time: 16.190574645996094 iteration: 1 best: 1
Fit A: Time: 15.691089153289795 Wasserstein Distance: [1.35952889919281, 1.0412681102752686, 0.957724392414093, 0.7217563986778259, 0.6938298344612122, 0.6721524000167847, 0.3683651387691498, 0.22064916789531708]
Fit spline Time: 0.397233247756958
logp: -4.089249134063721 -4.117161273956299 time: 16.164106130599976 iteration: 2 best: 2
Fit A: Time: 15.508404731750488 Wasserstein Distance: [0.6531975269317627, 0.6203418970108032, 0.6034573316574097, 0.592527449131012, 0.4831496775150299, 0.44546428322792053, 0.44303274154663086, 0.44046926498413086]
Fit spline Time: 0.377765417098999
logp: -1.6537156105041504 -1.6838488578796387 time: 15.95369863510131

Fit A: Time: 15.505736112594604 Wasserstein Distance: [0.24246646463871002, 0.22502559423446655, 0.15837320685386658, 0.11544281244277954, 0.10877730697393417, 0.09467226266860962, 0.0849934071302414, 0.07370493561029434]
Fit spline Time: 0.4656984806060791
logp: 8.479000091552734 8.457714080810547 time: 16.048293828964233 iteration: 25 best: 25
Fit A: Time: 16.890526056289673 Wasserstein Distance: [0.3133385479450226, 0.15220993757247925, 0.10352536290884018, 0.10036315023899078, 0.08899858593940735, 0.07628230005502701, 0.07103404402732849, 0.06806457787752151]
Fit spline Time: 0.5253996849060059
logp: 8.590267181396484 8.570430755615234 time: 17.507330417633057 iteration: 26 best: 26
Fit A: Time: 15.364413976669312 Wasserstein Distance: [0.3241828382015228, 0.16185122728347778, 0.09871261566877365, 0.0886971652507782, 0.07740549743175507, 0.06846388429403305, 0.06664247810840607, 0.06282997131347656]
Fit spline Time: 0.3685448169708252
logp: 8.697351455688477 8.677183151245117 time:

Fit spline Time: 0.3705456256866455
logp: 10.290056228637695 10.26437759399414 time: 15.559621572494507 iteration: 48 best: 48
Fit A: Time: 16.53456211090088 Wasserstein Distance: [0.3982549011707306, 0.22390864789485931, 0.06715776771306992, 0.05964124947786331, 0.05667900666594505, 0.054832663387060165, 0.04291391000151634, 0.04187938943505287]
Fit spline Time: 0.48751282691955566
logp: 10.346880912780762 10.317788124084473 time: 17.11902928352356 iteration: 49 best: 49
Fit A: Time: 18.399080276489258 Wasserstein Distance: [0.46354103088378906, 0.05812951549887657, 0.055591244250535965, 0.05506018176674843, 0.054916854947805405, 0.05162559077143669, 0.05002570524811745, 0.04404298961162567]
Fit spline Time: 0.3850522041320801
logp: 10.393630027770996 10.36898136138916 time: 18.858628511428833 iteration: 50 best: 50
Fit A: Time: 17.096572875976562 Wasserstein Distance: [0.477749228477478, 0.06224377080798149, 0.05488942191004753, 0.05462675541639328, 0.05170033872127533, 0.04563708975

Fit A: Time: 16.47503900527954 Wasserstein Distance: [0.4297577440738678, 0.04594510793685913, 0.0424349345266819, 0.04183749482035637, 0.0372372530400753, 0.035558391362428665, 0.032106198370456696, 0.03176924213767052]
Fit spline Time: 0.41387081146240234
logp: 11.317283630371094 11.20588493347168 time: 16.955049514770508 iteration: 72 best: 72
Fit A: Time: 16.433433532714844 Wasserstein Distance: [0.4279811680316925, 0.05232007056474686, 0.04158779978752136, 0.040267445147037506, 0.03947504609823227, 0.03490617498755455, 0.03355022147297859, 0.027833543717861176]
Fit spline Time: 0.37388086318969727
logp: 11.347772598266602 11.231697082519531 time: 16.875500679016113 iteration: 73 best: 73
Fit A: Time: 16.448989152908325 Wasserstein Distance: [0.42796802520751953, 0.05134221538901329, 0.04637492820620537, 0.03971016779541969, 0.03958171233534813, 0.038569338619709015, 0.03798011317849159, 0.030252523720264435]
Fit spline Time: 0.41774749755859375
logp: 11.377243995666504 11.25829505

Fit spline Time: 0.3723025321960449
logp: 12.034469604492188 11.844412803649902 time: 16.427862405776978 iteration: 95 best: 95
Fit A: Time: 16.0504047870636 Wasserstein Distance: [0.35590603947639465, 0.04700031504034996, 0.04049721360206604, 0.037672363221645355, 0.03283253684639931, 0.03206528350710869, 0.028753528371453285, 0.027792708948254585]
Fit spline Time: 0.40979981422424316
logp: 12.05809211730957 11.869550704956055 time: 16.536473512649536 iteration: 96 best: 96
Fit A: Time: 17.310365200042725 Wasserstein Distance: [0.3597193658351898, 0.03750203177332878, 0.033728696405887604, 0.03243871033191681, 0.03169853240251541, 0.02985413186252117, 0.028152236714959145, 0.026992149651050568]
Fit spline Time: 0.5657589435577393
logp: 12.077605247497559 11.890210151672363 time: 17.982255458831787 iteration: 97 best: 97
Fit A: Time: 17.11000370979309 Wasserstein Distance: [0.36313316226005554, 0.041946280747652054, 0.041595667600631714, 0.03484022617340088, 0.03185400366783142, 0.0307

Fit A: Time: 15.863288640975952 Wasserstein Distance: [0.35305675864219666, 0.03453575819730759, 0.03180399909615517, 0.031608518213033676, 0.029617035761475563, 0.02811018005013466, 0.025883425027132034, 0.01963474228978157]
Fit spline Time: 0.3822622299194336
logp: 12.540247917175293 12.285711288452148 time: 16.318023443222046 iteration: 119 best: 119
Fit A: Time: 16.192700147628784 Wasserstein Distance: [0.35497698187828064, 0.0327736996114254, 0.03256047144532204, 0.03245297819375992, 0.02940768003463745, 0.02642824314534664, 0.02058427594602108, 0.019060276448726654]
Fit spline Time: 0.3768634796142578
logp: 12.558114051818848 12.299209594726562 time: 16.640089750289917 iteration: 120 best: 120
Fit A: Time: 16.019155025482178 Wasserstein Distance: [0.35527166724205017, 0.03621112182736397, 0.02859485149383545, 0.02571912296116352, 0.022812698036432266, 0.022214701399207115, 0.01971145160496235, 0.0194203220307827]
Fit spline Time: 0.38631653785705566
logp: 12.573995590209961 12.30

Fit spline Time: 0.376572847366333
logp: 12.96964168548584 12.572311401367188 time: 13.711552858352661 iteration: 142 best: 142
Fit A: Time: 15.881656169891357 Wasserstein Distance: [0.30829259753227234, 0.14311721920967102, 0.028257757425308228, 0.023537764325737953, 0.02273186296224594, 0.022185368463397026, 0.022054562345147133, 0.020441332831978798]
Fit spline Time: 0.3734397888183594
logp: 12.972051620483398 12.587571144104004 time: 16.324626445770264 iteration: 143 best: 143
Fit A: Time: 16.027978658676147 Wasserstein Distance: [0.38387078046798706, 0.026193203404545784, 0.026161298155784607, 0.024456320330500603, 0.022849079221487045, 0.021855337545275688, 0.016325566917657852, 0.015378243289887905]
Fit spline Time: 0.3917558193206787
logp: 12.989790916442871 12.593791961669922 time: 16.49293613433838 iteration: 144 best: 144


KeyboardInterrupt: 

In [ ]:

logps_valid = model.evaluate_density(encoded_valid)
_ = plt.hist(logps_valid, bins=100, density=True, log=True, cumulative=True)
plt.show()
indices = np.argsort(logps_valid)


style_list=['dark_background']
# Plot a demonstration figure for every available style sheet.
for style_label in style_list:
    with plt.rc_context({"figure.max_open_warning": len(style_list)}):
        with plt.style.context(style_label):
            
            fig, ax = plt.subplots(5,5, figsize=(25,15))
            ax = ax.flatten()
            for nn, ii in enumerate(indices[0:25]):
                ax[nn].plot(np.squeeze(valid_data)[ii]*np.squeeze(valid_mask)[ii], )
                ax[nn].plot(np.squeeze(decoded_valid)[ii]*np.squeeze(valid_mask)[ii], color='orange')
                ax[nn].plot(np.squeeze(decoded_valid)[ii], color='red')
                ax[nn].text(0.05, 0.92, r'label=%s'%valid_labels[ii], fontsize=15, color='white',verticalalignment='top', horizontalalignment='left', transform=ax[nn].transAxes)
                ax[nn].text(0.05, 0.82, r'coords=(%.4f,%.4f)'%(valid_ras[ii],valid_decs[ii]), fontsize=15, color='white',verticalalignment='top', horizontalalignment='left', transform=ax[nn].transAxes)
                ax[nn].text(0.05, 0.72, r'z=%.3f'%valid_params[ii], fontsize=15, color='white',verticalalignment='top', horizontalalignment='left', transform=ax[nn].transAxes)
                ax[nn].text(0.05, 0.62, r'logp=%.2f'%logps_valid[ii], fontsize=15, color='white',verticalalignment='top', horizontalalignment='left', transform=ax[nn].transAxes)

plt.show()

In [ ]:
# cluster by recon error

In [ ]:
logps_train = model.evaluate_density(encoded_train)
_ = plt.hist(logps_train, bins=100, density=True, log=True, cumulative=True)
plt.show()
indices = np.argsort(logps_train)

style_list=['dark_background']
# Plot a demonstration figure for every available style sheet.
for style_label in style_list:
    with plt.rc_context({"figure.max_open_warning": len(style_list)}):
        with plt.style.context(style_label):
            
            fig, ax = plt.subplots(5,5, figsize=(25,15))
            ax = ax.flatten()
            for nn, ii in enumerate(indices[0:25]):
                ax[nn].plot(new_wl[:-1], np.squeeze(train_data)[ii]*np.squeeze(train_mask)[ii], )
                ax[nn].plot(new_wl[:-1], np.squeeze(decoded_train)[ii]*np.squeeze(train_mask)[ii], color='orange')
                ax[nn].plot(new_wl[:-1], np.squeeze(decoded_train)[ii], color='red')
                ax[nn].text(0.05, 0.92, r'z=%.2f'%train_params[ii], fontsize=15, color='white',verticalalignment='top', horizontalalignment='left', transform=ax[nn].transAxes)
                ax[nn].text(0.05, 0.82, r'logp=%.2f'%logps_train[ii], fontsize=15, color='white',verticalalignment='top', horizontalalignment='left', transform=ax[nn].transAxes)

plt.show()